In [64]:
from collections import deque
DEFAULT = 0
orcId = -2
danger = DEFAULT
m = DEFAULT
n = DEFAULT
gandalfX = DEFAULT
gandalfY = DEFAULT
gandorX = DEFAULT
gandorY = DEFAULT
ringsLoc = []
ringsDest = []
orcs = []

In [65]:
def isValid(row, col):
    return (row >= 0) and (row < m) and (col >= 0) and (col < n)

In [66]:
class Node:

    def __init__(self, x, y):

        self.x = x 
        self.y = y
        self.f = 0
        self.g = 0
        self.h = 0
        self.neighbors = []
        self.neighborsDirection = []
        self.previous = None
        self.orc = False
        self.rank = DEFAULT
        self.orcId = DEFAULT

    def orcIsHere(self):
            self.orc = True

    def rangeRank(self,rank):
        self.rank = rank
        
    def orcIdSet(self,id):
        self.orcId = id
    
    def getLoc(self):
        return self.x,self.y

    def addNeighbors(self, table, columns, rows):

        neighbor_x = self.x
        neighbor_y = self.y
    
        if neighbor_x < columns - 1:
            self.neighbors.append(table[neighbor_x+1][neighbor_y])
            self.neighborsDirection.append("L")
        if neighbor_x > 0:
            self.neighbors.append(table[neighbor_x-1][neighbor_y])
            self.neighborsDirection.append("R")
        if neighbor_y < rows -1:
            self.neighbors.append(table[neighbor_x][neighbor_y +1])
            self.neighborsDirection.append("U")
        if neighbor_y > 0: 
            self.neighbors.append(table[neighbor_x][neighbor_y-1])
            self.neighborsDirection.append("D")

In [67]:
class AStar:

    def __init__(self, cols, rows, start, end):

        self.cols = cols
        self.rows = rows
        self.start = start
        self.end = end
        

 
    def cleanOpenSet(openSet, currentNode):

        for i in range(len(openSet)):
            if openSet[i] == currentNode:
                openSet.pop(i)
                break

        return openSet

    def fillTable(table,rows, cols):

        for i in range(rows):
            for j in range(cols):
                table[i][j] = Node(i,j)
                if(orcs[i][j] < 0):
                    table[i][j].orcIsHere()
                    table[i][j].orcIdSet(orcs[i][j])
                if(orcs[i][j] > 0):
                    table[i][j].rangeRank(orcs[i][j])
        return table
 
    def heu(currentNode, end):

        distance =  abs(currentNode.x - end.x) + abs(currentNode.y - end.y)
        
        return distance

 
    def createTable(cols, rows):

        table = []
        for _ in range(cols):
            table.append([])
            for _ in range(rows):
                table[-1].append(0)
        
        return table


    def getNeighbors(table, cols, rows):
        for i in range(cols):
            for j in range(rows):
                table[i][j].addNeighbors(table, cols, rows)
        return table

    def startPath(openSet, closeSet, currentNode, end):
        global danger

        best_way = 0
        for i in range(len(openSet)):
            if openSet[i].f < openSet[best_way].f:
                best_way = i

        currentNode = openSet[best_way]
        final_path = []
        if currentNode == end:
            temp = currentNode
            while temp.previous:
                final_path.append(temp.previous)
                temp = temp.previous
            
            
        openSet = AStar.cleanOpenSet(openSet, currentNode)
        closeSet.append(currentNode)
        neighbors = currentNode.neighbors
        for neighbor in neighbors:
            if (neighbor in closeSet) or (neighbor.orc == True) or (danger+1 == neighbor.rank):
                continue
            else:
                temp_g = currentNode.g + 1
                control_flag = 0
                for k in range(len(openSet)):
                    if neighbor.x == openSet[k].x and neighbor.y == openSet[k].y:
                        if temp_g < openSet[k].g:
                            openSet[k].g = temp_g
                            openSet[k].h= AStar.heu(openSet[k], end)
                            openSet[k].f = openSet[k].g + openSet[k].h
                            openSet[k].previous = currentNode
                            if((openSet[k].orcId == currentNode.orcId) and (openSet[k].rank > 0) ):
                                danger = danger+1
                        else:
                            pass
                        control_flag = 1
  
                if control_flag == 1:
                    pass
                else:
                    neighbor.g = temp_g
                    neighbor.h = AStar.heu(neighbor, end)
                    neighbor.f = neighbor.g + neighbor.h
                    neighbor.previous = currentNode
                    openSet.append(neighbor)
        return openSet, closeSet, currentNode, final_path


In [68]:
def OrcsRange(orcs, x, y, rank,tablex,tabley):
    global orcId
    orcId = orcId - 1
    for i in range(x):
        for j in range():
            orcs[i][j] = 0

    for i in range(x - rank, x + rank + 1):
        if i in range(0,tabley):
            temp = rank - abs(i - y)
            for j in range(-temp, temp + 1):
                if (x + j) in range(0,tablex): 
                    orcs[x + j][i] = rank
    orcs[x][y] = orcId

    return orcs


In [69]:
def inputManager():
    global m, n
    global gandalfX , gandalfY ,gandorX,gandorY,orcs,ringsLoc,ringsDest
    m, n = map(int, input("n and m").split())
    gandalfX, gandalfY = map(int, input("start").split())
    gandorX, gandorY = map(int, input("end").split())
    k, l = map(int, input("k and l").split())
    orcs = [[0 for j in range(m)]
                    for j in range(n)]
    for j in range(k):
        x, y, lvl = map(int, input().split())
        orcs = OrcsRange(orcs, x, y, lvl,n,m)
    


    for j in range(l):
        x, y = map(int, input().split())
        ringsLoc.append([x, y])
    for j in range(l):
        x, y = map(int, input().split())
        ringsDest.append([x, y])
    return    


In [70]:
def main():
    inputManager()
    table = AStar.createTable(n, m)
    table = AStar.fillTable(table,n,m)
    table = AStar.getNeighbors(table, n, m)
    openSet  = []
    closeSet  = []
    currentNode = None
    final_path  = []
    openSet.append(table[gandalfX][gandalfY])
    end =table[gandorX][gandorY]
    while len(openSet) > 0:
        openSet, closeSet, currentNode, final_path = AStar.startPath(openSet, closeSet, currentNode,end)
        if len(final_path) > 0:
            break
        
    directions = []    
    for i in range(len(final_path)-1):
        temp = final_path[i+1]
        for j in range(len(final_path[i].neighbors)):
            if temp == final_path[i].neighbors[j]:
                print(final_path[i].neighborsDirection[j])
                break

main()